In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 

morph = MorphAnalyzer()

C:\Users\ksenia\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

## Данные

In [2]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return clean

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [9]:
habr_texts = open('habr_texts.txt', encoding = 'utf-8').read().splitlines()

In [12]:
import collections

In [15]:
f = open('habr_texts.vw', 'w', encoding='utf-8')

for i, text in enumerate(habr_texts):
    c = collections.Counter(text)
    doc = 'doc_'+ str(i) + ' '
    vw_text = ' '.join([x+':'+str(c[x]) for x in c])
    
    f.write(doc + vw_text  + '\n')
f.close()



В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [22]:
import pickle

In [16]:
texts = [clean(word_tokenize(text.lower())) for text in habr_texts]

In [37]:
texts[:10]

[['masstransit',
  'это',
  'open',
  'source',
  'библиотека',
  'разработать',
  'на',
  'язык',
  'для',
  '.net',
  'платформа',
  'упрощать',
  'работа',
  'шина',
  'дать',
  'который',
  'использоваться',
  'при',
  'построение',
  'распределенный',
  'приложение',
  'реализация',
  'soa',
  'service',
  'oriented',
  'architecture',
  'качество',
  'message',
  'broker',
  'мочь',
  'выступать',
  'rabbitmq',
  'azure',
  'service',
  'bus',
  'или',
  'in-memory',
  'менеджер',
  'случай',
  'in-memory',
  'область',
  'видимость',
  'ограничиваться',
  'процесс',
  'который',
  'проинициализировать',
  'экземпляр',
  '.содержание',
  'команда',
  'событиякоманда',
  'событие',
  'контракт',
  'сообщение',
  'роутингexchange',
  'формат',
  'сообщение',
  'консьюмер',
  'consumer',
  'конфигурация',
  'контейнер',
  'di',
  'наблюдатель',
  'observer',
  'новое',
  'masstransit',
  '3.0',
  'заключение',
  'опрос',
  'какой',
  '.net',
  'библиотека',
  'использовать',
  'вы',

In [24]:
with open('cleaned', 'wb') as fp:
    pickle.dump(texts, fp)

In [17]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [18]:
dictinary = gensim.corpora.Dictionary(texts)

In [19]:
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()

In [25]:
print(dictinary)

Dictionary(7649 unique tokens: ['.net', '2-х', '3.0', 'address', 'api']...)


4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем;

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

In [26]:
corpus = [dictinary.doc2bow(text) for text in texts]
# если текстов много, то тут может быть генератор

In [27]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, passes=1) # если поддерживается многопоточность


C:\Users\ksenia\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Посмотрим на топики: тут получилось не очень

In [38]:
#lda.print_topics()

Еще варианты

In [30]:
lda1 = gensim.models.LdaMulticore(corpus, 10, id2word=dictinary, passes=1) 


In [39]:
#lda1.print_topics()

In [34]:
lda2 = gensim.models.LdaMulticore(corpus, 5, id2word=dictinary, passes=3) 
#вот это выглядит неплохо

In [40]:
#lda2.print_topics()

In [35]:
lda3 = gensim.models.LdaMulticore(corpus, 7, id2word=dictinary, passes=10) 

In [41]:
#lda3.print_topics()

В целом тоже ничего, но странно, что темы так отличаются от тех, которые получились при разбивке на 5.

Мы, кажется, не убрали стоп-слова! Надо исправляться

In [48]:
import nltk
from nltk.corpus import stopwords

In [55]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ksenia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
len(texts)

4121

In [61]:
def stop_word(ttext):
    filtered_words = [word for word in ttext if word not in stopwords.words('russian')]
    return filtered_words

In [62]:
def all_stop(ttexts):
    return [stop_word(ttext) for ttext in ttexts]


In [63]:
cleaned = all_stop(texts)

In [64]:
with open('cleaned2', 'wb') as fp:
    pickle.dump(cleaned, fp)

In [65]:
dictinary2 = gensim.corpora.Dictionary(cleaned)

In [76]:
dictinary2.filter_extremes(no_above=0.3, no_below=25)
dictinary2.compactify()

In [77]:
print(dictinary2)

Dictionary(6589 unique tokens: ['.net', '2-х', '3.0', 'address', 'api']...)


In [78]:
corpus = [dictinary2.doc2bow(text) for text in cleaned]

In [73]:
lda4 = gensim.models.LdaMulticore(corpus, 10, id2word=dictinary, passes=3) # если поддерживается многопоточность


In [74]:
lda4.print_topics()

[(0,
  '0.026*"if" + 0.012*"gt" + 0.011*"private" + 0.010*"задержка" + 0.009*"продукт" + 0.009*"армия" + 0.009*"память" + 0.009*"python" + 0.007*"выявить" + 0.006*"research"'),
 (1,
  '0.006*"username" + 0.004*"истечение" + 0.004*"получение" + 0.004*"ок" + 0.003*"затем" + 0.003*"копирование" + 0.003*"сократиться" + 0.003*"отключаться" + 0.003*"нода" + 0.003*"отключение"'),
 (2,
  '0.005*"компонента" + 0.004*"сотрудник" + 0.003*"специальный" + 0.003*"постараться" + 0.003*"часы" + 0.003*"прийтись" + 0.002*"порту" + 0.002*"аккуратно" + 0.002*"бывать" + 0.002*"рождение"'),
 (3,
  '0.018*"бывать" + 0.007*"космос" + 0.003*"задний" + 0.003*"отметить" + 0.003*"частота" + 0.002*"новичок" + 0.002*"желать" + 0.002*"обзор" + 0.002*"обрабатывать" + 0.002*"переставать"'),
 (4,
  '0.008*"from" + 0.006*"v3" + 0.005*"увидеть" + 0.004*"ps" + 0.004*"удалить" + 0.004*"реальный" + 0.004*"simple" + 0.003*"значимый" + 0.003*"machine" + 0.003*"очевидно"'),
 (5,
  '0.006*"согласно" + 0.005*"бонус" + 0.005*"мин

как-то ниоч

In [79]:
lda5 = gensim.models.LdaMulticore(corpus, 7, id2word=dictinary, passes=5)
lda5.print_topics()

[(0,
  '0.027*"тайна" + 0.024*"if" + 0.017*"private" + 0.017*"extension" + 0.016*"устойчивый" + 0.015*"подобный" + 0.012*"сообщение" + 0.012*"неудобство" + 0.010*"увидеть" + 0.010*"ai"'),
 (1,
  '0.004*"компонента" + 0.003*"автоматизировать" + 0.002*"громкий" + 0.002*"mobile" + 0.002*"переставать" + 0.002*"сотрудник" + 0.002*"идеальный" + 0.002*"правило" + 0.002*"тормоз" + 0.002*"копирование"'),
 (2,
  '0.029*"бывать" + 0.011*"космос" + 0.005*"отметить" + 0.005*"видный" + 0.005*"задний" + 0.003*"доработать" + 0.003*"mode" + 0.003*"рождение" + 0.003*"верить" + 0.003*"отключение"'),
 (3,
  '0.084*"if" + 0.013*"private" + 0.013*"gt" + 0.010*"phone" + 0.009*"память" + 0.008*"значение" + 0.008*"state" + 0.008*"namespace" + 0.007*"армия" + 0.007*"upd"'),
 (4,
  '0.003*"разделение" + 0.003*"затем" + 0.003*"порту" + 0.003*"поздний" + 0.003*"получение" + 0.003*"выявить" + 0.003*"местами" + 0.003*"потенциальный" + 0.002*"показаться" + 0.002*"прийтись"'),
 (5,
  '0.007*"сократиться" + 0.007*"заде

In [80]:
lda5.print_topics()[:3]

[(0,
  '0.027*"тайна" + 0.024*"if" + 0.017*"private" + 0.017*"extension" + 0.016*"устойчивый" + 0.015*"подобный" + 0.012*"сообщение" + 0.012*"неудобство" + 0.010*"увидеть" + 0.010*"ai"'),
 (1,
  '0.004*"компонента" + 0.003*"автоматизировать" + 0.002*"громкий" + 0.002*"mobile" + 0.002*"переставать" + 0.002*"сотрудник" + 0.002*"идеальный" + 0.002*"правило" + 0.002*"тормоз" + 0.002*"копирование"'),
 (2,
  '0.029*"бывать" + 0.011*"космос" + 0.005*"отметить" + 0.005*"видный" + 0.005*"задний" + 0.003*"доработать" + 0.003*"mode" + 0.003*"рождение" + 0.003*"верить" + 0.003*"отключение"')]

Вроде ничего. Попробуем еще улучшить.

6) между словарем и обучением модели добавьте tfidf (gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

In [84]:
gensim.models.TfidfModel(corpus, id2word=dictinary); corpus = tfidf[corpus]

In [85]:
lda5 = gensim.models.LdaMulticore(corpus, 7, id2word=dictinary, passes=5)
lda5.print_topics()

[(0,
  '0.004*"си" + 0.004*"творить" + 0.003*"ужасно" + 0.003*"бонус" + 0.003*"проектировать" + 0.003*"игрушка" + 0.003*"несовместимый" + 0.003*"платить" + 0.003*"всерьёз" + 0.003*"площадь"'),
 (1,
  '0.010*"отдача" + 0.008*"очищать" + 0.007*"низкоуровневый" + 0.006*"тишина" + 0.006*"трёхмерный" + 0.006*"128" + 0.005*"пострадавший" + 0.004*"отключение" + 0.004*"течение" + 0.004*"закончить"'),
 (2,
  '0.009*"0." + 0.007*"ура" + 0.007*"сократиться" + 0.005*"донести" + 0.005*"callback" + 0.005*"постараться" + 0.004*"эффективность" + 0.004*"физически" + 0.004*"ценник" + 0.004*"стрелять"'),
 (3,
  '0.019*"if" + 0.010*"phone" + 0.008*"армия" + 0.007*"отсчёт" + 0.006*"azure" + 0.006*"state" + 0.005*"private" + 0.005*"нить" + 0.005*"устойчивый" + 0.005*"python"'),
 (4,
  '0.010*"swift" + 0.009*"воздух" + 0.008*"качественно" + 0.008*"распространяться" + 0.006*"ткань" + 0.005*"указание" + 0.004*"-r" + 0.004*"синий" + 0.004*"логирование" + 0.003*"слава"'),
 (5,
  '0.002*"бывать" + 0.001*"космос" 

как будто стало только хуже....

In [86]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [88]:
stexts = [' '.join(text) for text in cleaned]

In [89]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

In [90]:
model = NMF(n_components=30)

In [91]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [93]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [94]:
get_nmf_topics(model, 10) 

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,продукт,gt,игра,марс,объект,сеть,камера,js,ракета,файл,...,язык,книга,public,центр,бот,робот,товар,звук,if,сайт
1,сотрудник,lt,игрок,земля,значение,трафик,устройство,react,спутник,docker,...,программирование,часы,string,дата,telegram,ребёнок,скидка,сигнал,int,страница
2,бизнес,div,игровой,планета,элемент,связь,смартфон,css,ступень,sudo,...,php,каковать,void,облачный,сообщение,автомобиль,магазин,наушник,amp,реклама
3,клиент,class,играть,космический,блок,оператор,аккумулятор,javascript,двигатель,php,...,программист,порекомендовать,new,инфраструктура,телеграм,lego,цена,усилитель,lt,браузер
4,менеджер,name,vr,луна,метод,ip,телефон,angular,spacex,http,...,java,профессиональный,this,услуга,чат,робототехника,рубль,частота,return,контент
5,заказчик,this,персонаж,аппарат,строка,интернет,дисплей,веб,космический,сервер,...,лекция,каким,return,облако,bot,машина,россия,звуковой,else,клиент
6,программист,props,steam,орбита,класс,устройство,видео,vue,пуск,nginx,...,программа,предпочитать,private,цод,api,датчик,российский,музыка,null,домен
7,crm,html,unity,астероид,запрос,dpi,экран,браузер,орбита,скрипт,...,перевод,путь,var,оборудование,мессенджер,движение,покупка,искажение,char,google
8,деньга,return,геймплей,солнечный,точка,адрес,usb,компонент,носитель,etc,...,курс,слушать,класс,сервер,канал,беспилотный,рынок,акустический,void,сервер
9,опыт,function,движок,станция,текстура,канал,корпус,dom,корабль,папка,...,python,бумажный,class,сервис,message,дрон,налог,напряжение,std,рекламный


In [95]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=6, max_df=0.2, lowercase=False)
X = vectorizer.fit_transform(stexts)

In [100]:
model = NMF(n_components=30)
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [101]:
get_nmf_topics(model, 10) 

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,сотрудник,lt,игра,робот,процессор,запрос,камера,космический,android,js,...,бот,пациент,php,центр,public,звук,if,vr,российский,резервный
1,бизнес,div,игрок,ребёнок,intel,страница,смартфон,спутник,google,react,...,telegram,мозг,docker,дата,string,сигнал,int,виртуальный,рынок,виртуальный
2,программист,class,игровой,lego,память,тест,видео,орбита,ios,css,...,сообщение,учёный,nginx,услуга,this,наушник,amp,реальность,google,вм
3,менеджер,name,играть,датчик,диск,строка,регистратор,ракета,мобильный,javascript,...,чат,клетка,http,инфраструктура,void,усилитель,return,oculus,страна,машина
4,заказчик,props,персонаж,робототехника,ядро,таблица,телефон,марс,app,angular,...,телеграм,заболевание,sudo,облачный,new,частота,else,шлем,россия,бэкап
5,ит,this,steam,автомобиль,гб,текст,карта,аппарат,swift,веб,...,bot,исследование,etc,оборудование,return,звуковой,null,vive,реклама,veeam
6,зарплата,html,unity,машина,ssd,тестирование,экран,земля,play,vue,...,api,болезнь,ip,цод,private,напряжение,char,дизайн,закон,диск
7,crm,std,геймплей,движение,производительность,блок,аккумулятор,луна,view,браузер,...,канал,ген,usr,облако,class,искажение,void,htc,налог,копирование
8,деньга,type,жанр,дрон,ноутбук,запись,дисплей,станция,gradle,html,...,мессенджер,врач,168,провайдер,класс,музыка,const,unreal,доход,восстановление
9,специалист,input,движок,education,накопитель,api,корпус,наса,activity,компонент,...,message,лечение,контейнер,ит,var,ток,end,дополнить,рубль,копия
